In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/mice-protein-expression/Data_Cortex_Nuclear.csv')

In [ ]:
 data.head()

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
print(data.columns)

In [ ]:
data_strings = data.select_dtypes(include = object)
print(data_strings.columns)

In [ ]:
data = data.drop(['MouseID','Genotype', 'Treatment', 'Behavior'], axis = 1)

In [ ]:
data.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
x_data = data[data.columns[0:len(data.columns) -1 ]]

In [ ]:
y_data = data[data.columns[len(data.columns)-1]]

In [ ]:
x_show = np.asarray(x_data)
fig , axes = plt.subplots(np.size(x_show, 1), 1, figsize = (8, 8*np.size(x_show,1)))
for i in range(0 , np.size(x_show, 1)):
    axes[i].set_xlabel(list(x_data.columns)[i], color = 'green', fontsize = 15)
    axes[i].set_ylabel('disturbution', color = 'green', fontsize = 15)
    axes[i].set_title(list(x_data.columns)[i]+'\'s disturbution', color = 'yellow', fontsize = 20)
    axes[i].grid('True')
    axes[i].hist(x_show[:, i])
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(y_data)

In [ ]:
plt.figure(figsize = (8,8))
plt.hist(y_data, rwidth = 0.3)
plt.xlabel('class of mice')
plt.ylabel('no of samples')
plt.title('disturbution of classes of mices')
plt.show()

In [ ]:
x_data = x_data.interpolate()

In [ ]:
x_data.isna().sum().sort_values(ascending = False)

In [ ]:
x_data.BCL2_N = x_data.BCL2_N.fillna(x_data.BCL2_N.mean())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x_data)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
select_k_best = SelectKBest(chi2, 50)
select_k_best.fit(x_scaled, y_data)
print(select_k_best.scores_)

In [ ]:
x_scaled.shape

In [ ]:
x_compute = x_data[x_data.columns[select_k_best.scores_>20]]

In [ ]:
len(x_compute.columns)

In [ ]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(50, activation='relu', input_shape=(21,)))
model.add(layers.Dense(14, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.utils.np_utils import to_categorical
y_data = to_categorical(y_data)

In [ ]:
history = model.fit(x_compute, y_data,epochs=50,validation_split = 0.25)

In [ ]:
hist_dict = history.history
loss_values = hist_dict['loss']
val_loss_values = hist_dict['val_loss']
epochs = range(1, 51)
plt.figure(figsize = (10, 4))
plt.plot(epochs, loss_values, 'bo', label = 'training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
from keras.utils import plot_model
import pydot
plot_model(model, to_file ='model_plot.png', show_shapes=True, show_layer_names=True)